In [ ]:
# Kiểm tra nếu trạng thái hiện tại đã đạt đến mục tiêu (tất cả đĩa ở cọc C, xếp đúng thứ tự)

def la_trang_thai_dich(trang_thai, so_dia):
    return trang_thai[2] == tuple(range(so_dia, 0, -1))

# Tạo các trạng thái kế tiếp bằng cách di chuyển đĩa từ một cọc sang cọc khác
def tao_lan_can(trang_thai):
    danh_sach_lan_can = []
    for i in range(3):  # 3 cọc: A, B, C
        if not trang_thai[i]:
            continue
        dia = trang_thai[i][-1]
        for j in range(3):
            if i == j:
                continue
            if not trang_thai[j] or trang_thai[j][-1] > dia:
                trang_thai_moi = [list(coc) for coc in trang_thai]
                trang_thai_moi[i].pop()
                trang_thai_moi[j].append(dia)
                danh_sach_lan_can.append((tuple(tuple(coc) for coc in trang_thai_moi), (i, j)))
    return danh_sach_lan_can

# Hàm heuristic: đếm số đĩa sai vị trí ở cọc C

def uoc_luong_sai_o_coc_C(trang_thai, so_dia):
    dich = list(range(so_dia, 0, -1))
    coc_C = list(trang_thai[2])
    sai = 0
    for i in range(len(coc_C)):
        if i >= len(dich) or coc_C[-1 - i] != dich[i]:
            sai += 1
    return sai

# Thuật toán A*

def a_sao(trang_thai_ban_dau, so_dia):
    MO = [trang_thai_ban_dau]
    DONG = [] #Lưu trữ những đỉnh đã được xét đến
    f = {trang_thai_ban_dau: uoc_luong_sai_o_coc_C(trang_thai_ban_dau, so_dia)} #Chi phí của đỉnh
    g = {trang_thai_ban_dau: 0} #Chi phí từ trạng thái ban đầu tới các trạng thái khác
    parent = {}

    while MO:
        min_f = float('inf')
        n = None
        for trang_thai in MO:
            if f[trang_thai] < min_f:
                min_f = f[trang_thai]
                n = trang_thai

        if la_trang_thai_dich(n, so_dia):
            return reconstruct_path(n, parent, g, so_dia)

        MO.remove(n)
        DONG.append(n)

        for lan_can, buoc_di in tao_lan_can(n):
            cost_new = g[n] + 1
            if lan_can not in g or cost_new < g[lan_can]:
                g[lan_can] = cost_new
                f[lan_can] = g[lan_can] + uoc_luong_sai_o_coc_C(lan_can, so_dia)
                parent[lan_can] = (n, buoc_di)

            if lan_can not in MO and lan_can not in DONG:
                MO.append(lan_can)

    return None

# Hàm tái tạo đường đi từ trạng thái đích về gốc
def reconstruct_path(state, parent, g, so_dia):
    path = []
    while state in parent: # Lặp ngược từ trạng thái hiện tại về trạng thái gốc
        prev_state, move = parent[state] # Lấy trạng thái cha và bước di chuyển để đến state.
        h_val = uoc_luong_sai_o_coc_C(state, so_dia)
        f_val = g[state] + h_val
        path.append((move, state, g[state], h_val, f_val)) # Lưu lại thông tin từng bước
        state = prev_state #Truy ngược lại cha
    path.reverse()
    return path

# Hàm in ra lộ trình và mã hóa trạng thái
def in_lo_trinh(duong_di, so_dia):
    print(f" Giải pháp tìm thấy với {len(duong_di)} bước:\n")
    for i, (buoc, trang_thai, g, h, f_val) in enumerate(duong_di):
        tu, den = buoc
        ten_coc = ['A', 'B', 'C']
        print(f"Bước {i+1}: Di chuyển từ cọc {ten_coc[tu]} sang cọc {ten_coc[den]}")
        A, B, C = [list(coc) for coc in trang_thai]
        print(f"  Trạng thái: A: {A} | B: {B} | C: {C}")

        trang_thai_ma_hoa = [0] * so_dia
        for coc, vi_tri in zip([A, B, C], [1, 2, 3]):
            for dia in coc:
                trang_thai_ma_hoa[dia - 1] = vi_tri
        print(f"  Trạng thái mã hóa: {trang_thai_ma_hoa}")
        print(f"  → Chi phí g(n) = {g}, Heuristic h(n) = {h}, Tổng f(n) = {f_val}\n")

    print(f" Tổng chi phí: {len(duong_di)}")

# Hàm main
def chay_chuong_trinh():
    try:
        so_dia = int(input("Nhập số đĩa (n ≥ 1): "))
        if so_dia < 1:
            raise ValueError
    except ValueError:
        print("Giá trị không hợp lệ. Vui lòng nhập số nguyên ≥ 1.")
        return

    trang_thai_ban_dau = (tuple(range(so_dia, 0, -1)), tuple(), tuple())
    print("\nTìm giải pháp bằng thuật toán A*\n")
    duong_di = a_sao(trang_thai_ban_dau, so_dia)

    if duong_di:
        in_lo_trinh(duong_di, so_dia)
    else:
        print("Không tìm thấy giải pháp.")

if __name__ == "__main__":
    chay_chuong_trinh()
